In [1]:
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from clustertools import *

In [2]:
dataset_file = "datasets/fr_dataset.csv"

In [3]:
with open(dataset_file, "r") as f:
    dataset = f.read().splitlines()

In [4]:
# Séparation tweets / tags
dataset = [line.split(',') for line in dataset]

In [5]:
# Séparation des twetts en listes de mots
dataset = [[line[0].split(), line[1]] for line in dataset]

In [6]:
# Modification des tags en 0 si le tag contient normal et 1 sinon.
dataset = [[line[0], 0] if "normal" in line[1] else [line[0], 1] for line in dataset]

In [7]:
tweets = [line[0] for line in dataset]
wlist = create_sorted_wlist(tweets)

In [8]:
word_index = {'<pad>':0, '<unknown>':1}
for i, w in enumerate(wlist):
    word_index[w[0]] = i + 2

In [9]:
# Création des jeu de données train / test
dataset_train, dataset_test = train_test_split(dataset, test_size=0.2)

In [10]:
x_train_raw = [line[0] for line in dataset_train]
y_train = [line[1] for line in dataset_train]
x_test_raw = [line[0] for line in dataset_test]
y_test = [line[1] for line in dataset_test]

In [11]:
x_train = [[word_index[w] for w in line] for line in x_train_raw]
x_test = [[word_index[w] for w in line] for line in x_test_raw]

In [56]:
max_len = max([len(t) for t in dataset_split_tweets])

In [73]:
max_len

28

In [12]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                     value   = 0,
                                                     padding = 'post',
                                                     maxlen  = 28)

x_test  = keras.preprocessing.sequence.pad_sequences(x_test,
                                                     value   = 0 ,
                                                     padding = 'post',
                                                     maxlen  = 28)

In [13]:
x_traina = np.array(x_train)
y_traina = np.array(y_train)
x_testa = np.array(x_test)
y_testa = np.array(y_test)

In [45]:
model = keras.Sequential()
model.add(keras.layers.Input(shape = (28,) ))
model.add(keras.layers.Embedding(input_dim = 10958, output_dim = 512, input_length = 28))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(256, activation = "relu"))
#model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))


In [46]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [47]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 28, 512)           5610496   
                                                                 
 global_average_pooling1d_6   (None, 512)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dense_14 (Dense)            (None, 1)                 257       
                                                                 
Total params: 5,742,081
Trainable params: 5,742,081
Non-trainable params: 0
_________________________________________________________________


In [48]:
savemodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath="validation_model", verbose=0, save_best_only=True)

In [49]:
%%time

history = model.fit(x_traina,
                    y_traina,
                    epochs          = 30,
                    batch_size      = 512,
                    validation_data = (x_testa, y_testa),
                    verbose         = 1,
                    callbacks       = [savemodel_callback])

Epoch 1/30
7/7 [==============================] - 2s 226ms/step - loss: 0.6556 - accuracy: 0.6459 - val_loss: 0.5890 - val_accuracy: 0.7472
Epoch 2/30
7/7 [==============================] - 1s 202ms/step - loss: 0.5914 - accuracy: 0.7132 - val_loss: 0.5619 - val_accuracy: 0.7472
Epoch 3/30
7/7 [==============================] - 1s 200ms/step - loss: 0.5874 - accuracy: 0.7132 - val_loss: 0.5588 - val_accuracy: 0.7472
Epoch 4/30
7/7 [==============================] - 0s 60ms/step - loss: 0.5697 - accuracy: 0.7132 - val_loss: 0.5620 - val_accuracy: 0.7472
Epoch 5/30
7/7 [==============================] - 1s 223ms/step - loss: 0.5572 - accuracy: 0.7132 - val_loss: 0.5553 - val_accuracy: 0.7472
Epoch 6/30
7/7 [==============================] - 1s 199ms/step - loss: 0.5343 - accuracy: 0.7150 - val_loss: 0.5468 - val_accuracy: 0.7484
Epoch 7/30
7/7 [==============================] - 1s 197ms/step - loss: 0.5027 - accuracy: 0.7303 - val_loss: 0.5429 - val_accuracy: 0.7497
Epoch 8/30
7/7 [=====

In [51]:
mod = keras.models.load_model('validation_model')

In [54]:
with open('datasets/MillionTweetsMinStp_NEFU.txt', 'r') as f:
    dataset_1M = [line.split() for line in f.read().splitlines()]

In [60]:
dataset_1M_enc = [[word_index[w] if w in word_index else 1 for w in line ] for line in dataset_1M]

In [67]:
dataset_1M_enc = keras.preprocessing.sequence.pad_sequences(dataset_1M_enc,
                                                     value   = 0,
                                                     padding = 'post',
                                                     maxlen  = 28)

In [68]:
dataset_1M_enca = np.array(dataset_1M_enc)

In [69]:
pred = mod.predict(dataset_1M_enca[:50])

In [71]:
testpred = mod.predict(x_testa)

In [85]:
for i, elt in enumerate(testpred):
    if (elt >= 0.5 and y_test[i] == 1):
        print(' '.join(x_test_raw[i]))

@user ferme ta grande gueule une deuxième fois si tu voulais parler à ta sale arabe de copine fallait la mentionner elle et pas moi.
@user j’ai vu quelques renois la pépom j’espère qu’un va la gérer pour qu’elle ne tweet plus jamais ces conneries
@user la « jeune fille moderne » à la fois pour l’avortement le plus libre contre la peine de mort (pour des c… @url
@user @user mytho t'as juste voulu te faire ken par les renois de ton lycée ils ont pas voulu de to… @url
ce serait bien que norman fasse une vidéo contre l’avortement
@user ce qui choque en fait c'est que vous ne parlez que des noirs ... quid des asiats? des rebeus? des indiens? etc ...
@user @user mais le renois c’est pas le pote de jaymax là ?
soeur elle sept ans lui apprend jouer au echec elle nous dis mais pourquoi faut proteger le roi ? c'est une gauchiste certifiee
@user les gauchistes légifères pour protéger leurs racailles et migrants adorés les gilets les uritrottoir… @url
@user @user j'ai pas dit que t'écrivais à plac

In [84]:
' '.join(dataset_1M[0])

'commence stream! va modéliser ( ) fakemons serveur pixelmon ( commission )'